In [ ]:
System.out.println("Hello World");

In [ ]:
%loadFromPOM pom.xml

In [ ]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.IGenericClient;
import ca.uhn.fhir.rest.api.MethodOutcome;
import org.hl7.fhir.r4.model.*;

In [ ]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

In [ ]:
Bundle patients = client.search()
        .forResource(Patient.class)
        .returnBundle(Bundle.class)
        .execute();

for(Bundle.BundleEntryComponent entry : patients.getEntry()) {
    Patient patient = (Patient) entry.getResource();
    HumanName name = patient.getName().get(0);
    // Print the patient's family name and given name(s)
    System.out.println( name.getFamily() +" "+ name.getGivenAsSingleString());
}

In [ ]:

patients = client.search()
        .forResource(Patient.class)
        // sort by family name ascending, then given name ascending. 
        .sort().ascending(Patient.BIRTHDATE)
        .returnBundle(Bundle.class)
        .execute();

for(Bundle.BundleEntryComponent entry : patients.getEntry()) {
    Patient patient = (Patient) entry.getResource();
    HumanName name = patient.getName().get(0);
    System.out.println( name.getFamily() + " " + name.getGivenAsSingleString() + " " + patient.getBirthDate());
}

In [ ]:
// Perform a search
Bundle results = client.search()
      .forResource(Patient.class)
      .where(Patient.FAMILY.matches().value("陳"))
      .returnBundle(Bundle.class)
      .execute();

System.out.println("Found " + results.getEntry().size() + " patients named '陳'");


In [ ]:
//Create a new patient, add name, address, birthday on 1969-04-01 and then save it to the server
Patient newPatient = new Patient();
newPatient.addName().setFamily("陳").addGiven("大明");
newPatient.addAddress().setCity("台北市").setCountry("台灣");
newPatient.setBirthDateElement(new DateType("1939-04-01"));

// Invoke the server create method (and send pretty-printed JSON
// encoding to the server
MethodOutcome  outcome = client.create()
    .resource(newPatient)
    .prettyPrint()
    .encodedJson()
    .execute();
    
System.out.println("Got ID: " + outcome.getId());